In [ ]:
%pip install tensorflow

In [104]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np  
import csv
from functions import connect, get_array, create_data
data_client = connect()

ImportError: cannot import name 'create_t_and_t' from 'functions' (c:\Users\HP\Desktop\Other\Alpaca\Alpaca_stock_data\functions.py)

In [ ]:
stock = "AMD"

'''below are finding for the number of when create_data works and doesnt 6/24'''
#create_data("SPY", 1055, data_client)    # on 6/24 (monday) test: 100- 106 works. 107-111 doesn't work
# 112 - 113 works   114 - 118 doesnt work     | 119 - 120 works (swaps a few more times very quickly?)

create_data(stock, 1055, data_client)

In [ ]:
SPY_array = get_array("SPY")
stock_array = get_array(stock)

In [99]:
import random
def create_t_and_t(num, stock_array, SPY_array):
    '''Creates the input for a NN x_,y_traing and x_,y_test. right now randomly selects from 80% sampling instead of a single run through all -- could change'''
    x_train = np.zeros((num, 14))
    y_train = np.zeros(num)
    x_test = np.zeros((len(stock_array) - int(len(stock_array)*.8), 14))
    y_test = np.zeros(len(stock_array) - int(len(stock_array)*.8))



    for i in range(num):
        random_num = random.randint(0, int(len(stock_array)*.8) - 8)
        x_train[i, :7] = SPY_array[random_num:random_num + 7]
        x_train[i, 7:14] = stock_array[random_num:random_num + 7]
        if (stock_array[random_num+8] > 0.0): 
            y_train[i] = 1
        else:
            y_train[i] = 0
    for i in range(int(len(stock_array) *.8), len(stock_array)):
        k = i - int(len(stock_array) *.8)
        x_test[k, :7] = SPY_array[k:k + 7]
        x_test[k, 7:14] = stock_array[k:k + 7]
        if (stock_array[k+8] > 0.0): 
            y_test[k] = 1
        else:
            y_test[k] = 0
    return x_train, y_train, x_test, y_test

In [101]:


x_train, y_train, x_test, y_test = create_t_and_t(2050, stock_array, SPY_array)
print(x_train.shape)   # train on x percent of stocks?
# print(x_train)
print(y_train.shape)
# print(y_train)



(2050, 14)
(2050,)


In [ ]:
# num_samples = 10000
# num_features = 14
# num_classes = 3

# x_train = np.random.rand(num_samples, num_features)
# y_train = np.random.randint(num_classes, size=num_samples)

# x_test = np.random.rand(int(num_samples * 0.2), num_features)
# y_test = np.random.randint(num_classes, size=int(num_samples * 0.2))

# print("Shape of x_train:", x_train.shape)
# print("Shape of y_train:", y_train.shape)
# print("Shape of x_test:", x_test.shape)
# print("Shape of y_test:", y_test.shape)

In [ ]:
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(14,)),   # changed 100 from num samples 
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))  # change back the validation_data x_train to x_test same for y_train


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")


In [ ]:
# running once
new_input = np.array([x_train[0]])  # 1 sample, 14 features

print(new_input)
print(np.array([x_train[1]]))

#new_input = np.array(x_train[1])
# Make a prediction
prediction = model.predict(new_input)
print(prediction)

In [ ]:
model.save('synthetic_model.h5')
